In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

Load, clean, and write back to disk.

In [2]:
# Read files from directory.
files = [x for x in os.listdir('data') if '_data_only' in x]

# Sort by earlier year first.
files = list(sorted(files, key=lambda x: int(x[:4])))

print('File names:', files)

# Load into memory.
dfs = [
    pd.read_csv('data/' + file_name, delimiter=',', quotechar='"', encoding='ISO-8859-1')
    for file_name in files
]

# Clean up column names for consistency. 
for i, dframe in enumerate(dfs):
    dfs[i].columns = [
        re.sub(r"_+", "_", x \
            .strip() \
            .lower() \
            .replace(' ', '_') \
            .replace('\n', '_') \
            .replace('ï»¿agency', 'agency')
        )
        for x in dframe.columns
    ]

# Check for columns that are not shared between dataframes.
cols = [set(dframe.columns) for dframe in dfs]

cols_not_shared = []
common_columns_across_dfs = set.intersection(*cols)
for i, col_set in enumerate(cols):
    other_cols = set.union(*(cols[: i] + cols[i + 1 :])) 
    unique_to_this_df = col_set - other_cols  
    missing_from_this_df = other_cols - col_set  
    cols_not_shared.extend(unique_to_this_df)

for i, df in enumerate(dfs):
    for col in cols_not_shared:
        if col in df.columns:
            print(f'Dropping {col} because col is not shared.')
            dfs[i] = df.drop(col, axis=1)

# Add year to each dataframe.
for i, year in enumerate(range(2018, 2023)):
    dfs[i]['year'] = year

# Combine into one dataframe.
df = pd.DataFrame()
for dframe in dfs:
    df = pd.concat([df, dframe], axis=0)

len_df = len(df)
df = df[df['any_data_questionable?'] != 'Yes']
print(f'Dropped {len_df - len(df)} rows because data was questionable.')

# Drop cols with many NaN values.
MIN_NANS_PERCENT = .15  # Each year is roughly 20% of the total rows
for col in df.columns:
    if df[col].isna().sum() / len(df) > MIN_NANS_PERCENT:
        print(f'Dropping {col} because % NaN is: {df[col].isna().sum() / len(df)}')
        df = df.drop(col, axis=1)

# Remove extra characters from numeric columns and cast to float.
numeric_cols = [
    'cost_per_hour',
    'primary_uza_population',
    'vehicle_revenue_miles',
    'passenger_miles',
    'vehicle_revenue_hours',
    'unlinked_passenger_trips',
    'total_operating_expenses',
    'fare_revenues_earned',
    'cost_per_passenger_mile',
    'cost_per_passenger',
    'cost_per_hour',
    'fare_revenues_per_total_operating_expense_(recovery_ratio)',
    'primary_uza_population',
    'agency_voms',
    'mode_voms',
    'fare_revenues_per_unlinked_passenger_trip'
]

for col in numeric_cols:
    df[col] = df[col].apply(lambda x: (
        str(x) \
            .replace('$', '') \
            .replace('(', '') \
            .replace(')', '') \
            .replace(',', '')
            .strip()
    ))
    df[col] = df[col].astype(float)

df.reset_index(drop=True, inplace=True)

display(df.sample(1, random_state=538))

File names: ['2018_Metrics_data_only.csv', '2019_Metrics_data_only.csv', '2020-Metrics_data_only.csv', '2021 Metrics_static_data_only.csv', '2022_NTD_Annual_Data_-_Metrics_20231102_data_only.csv']
Dropping 0 because col is not shared.
Dropping uace_code because col is not shared.
Dropping uza_name because col is not shared.
Dropping passengers_per_vehicle_revenue_hour because col is not shared.
Dropped 443 rows because data was questionable.
Dropping legacy_ntd_id because % NaN is: 0.2714436015283238
Dropping ratios: because % NaN is: 1.0
Dropping fare_revenues_per_unlinked_passenger_trip_questionable because % NaN is: 0.9957361980176089
Dropping fare_revenues_per_total_operating_expense_(recovery_ratio)_questionable because % NaN is: 1.0
Dropping cost_per_hour_questionable because % NaN is: 0.9985602746552965
Dropping passengers_per_hour because % NaN is: 0.20438562489617365
Dropping passengers_per_hour_questionable because % NaN is: 0.9951270834486959
Dropping cost_per_passenger_ques

,agency,city,state,ntd_id,organization_type,reporter_type,primary_uza_population,agency_voms,mode,tos,mode_voms,fare_revenues_per_unlinked_passenger_trip,fare_revenues_per_total_operating_expense_(recovery_ratio),cost_per_hour,cost_per_passenger,cost_per_passenger_mile,fare_revenues_earned,total_operating_expenses,unlinked_passenger_trips,vehicle_revenue_hours,passenger_miles,vehicle_revenue_miles,year
8777,Fairmont Marion County Transit Authority,Fairmont,WV,3R05-30149,Independent Public Agency or Authority of Tran...,Rural Reporter,0.00,23.00,MB,DO,12.00,0.66,0.07,107.31,10.11,0.00,98648.00,1508329.00,149140.00,14056.00,0.00,282978.00,2020


In [3]:
def drop_cols(col, df):
    df_len = len(df)
    df = df.dropna(subset=[col])
    print(f'Dropped {df_len - len(df)} rows for having null {col}')
    return df

df = drop_cols('city', df)
df = drop_cols('passenger_miles', df)
df = drop_cols('cost_per_passenger_mile', df)

Dropped 64 rows for having null city
Dropped 0 rows for having null passenger_miles
Dropped 0 rows for having null cost_per_passenger_mile


In [4]:
df \
    .loc[:, ['city', 'year', 'mode', 'passenger_miles']] \
    .sort_values('passenger_miles', ascending=False).head(10)

,city,year,mode,passenger_miles
3638,New York,2019,HR,10462782577.00
0,New York,2018,HR,9989099073.00
14374,Brooklyn,2022,HR,7055402031.00
10786,Brooklyn,2021,HR,5668693486.00
7231,Brooklyn,2020,HR,4676670633.00
3748,Jamaica,2019,CR,3929859957.00
116,Jamaica,2018,CR,3405961936.00
93,New York,2018,CR,2154521183.00
12,Newark,2018,CR,2148639449.00
3732,New York,2019,CR,2034489613.00


In [5]:
df \
    .loc[:, ['city', 'year', 'mode', 'passenger_miles']] \
    .sort_values('passenger_miles', ascending=False).tail(10)

,city,year,mode,passenger_miles
7680,Nashville,2020,DR,0.00
7694,Pine Bluff,2020,DR,0.00
7711,Mount Pleasant,2020,DR,0.00
7714,Mount Pleasant,2020,MB,0.00
7715,Frederick,2020,DR,0.00
7716,Centralia,2020,DR,0.00
7718,Marshall,2020,MB,0.00
7722,Centralia,2020,MB,0.00
7723,Marshall,2020,DR,0.00
18058,Jackson,2022,DR,0.00


In [6]:
df \
    .loc[:, ['city', 'year', 'mode', 'cost_per_passenger_mile']] \
    .sort_values('cost_per_passenger_mile', ascending=False)

,city,year,mode,cost_per_passenger_mile
14944,Modesto,2022,CB,3056.03
7365,Woodbridge,2020,DR,963.30
13834,Detroit,2021,SR,802.43
12059,Minneapolis,2021,DR,653.42
15106,Oklahoma City,2022,FB,244.35
...,...,...,...,...
7716,Centralia,2020,DR,0.00
7718,Marshall,2020,MB,0.00
7722,Centralia,2020,MB,0.00
7723,Marshall,2020,DR,0.00


In [7]:
# Check Modesto Commuter bus outliers
display(df[(df.city == 'Modesto') & (df['mode'] == 'CB')] \
    .loc[:, ['city', 'year', 'cost_per_passenger_mile']] )

# Drop Modesto Commuter bus
df = df[(df.city != 'Modesto') | (df['mode'] != 'CB')]

,city,year,cost_per_passenger_mile
1363,Modesto,2018,0.00
4971,Modesto,2019,0.00
8510,Modesto,2020,0.00
12049,Modesto,2021,0.00
14944,Modesto,2022,3056.03


In [8]:
len_df = len(df)
dropped_rows = df[df.cost_per_passenger_mile == 0].copy()
df = df[df.cost_per_passenger_mile > 0]
print(f'Dropped {len_df - len(df)} rows for having zero cost per passenger mile')
print(f'Breakdown of dropped rows:')
dropped_rows.groupby(['year', 'mode']).size()

Dropped 11882 rows for having zero cost per passenger mile
Breakdown of dropped rows:


year  mode
2018  CB        96
      DR      1440
      DT        31
      FB        10
      MB       771
      RB         1
      TR         1
      VP        26
2019  CB        88
      DR      1434
      DT        32
      FB        12
      MB       776
      RB         1
      TR         1
      VP        24
2020  CB        85
      DR      1491
      FB        11
      MB       779
      RB         1
      SR         1
      TR         1
      VP        25
2021  CB        84
      CC         1
      DR      1499
      FB        14
      MB       763
      MG         2
      RB         1
      SR         2
      TR         2
      VP        24
2022  CB        79
      DR      1480
      FB        12
      IP         1
      MB       755
      RB         1
      SR         1
      TR         1
      VP        22
dtype: int64

In [9]:
for year in df.year.unique():
    for col in df.columns:
        is_na = df[df.year == year][col].isna().sum()
        is_na_percent = round(is_na / len(df[df.year == year]) * 100, 2)
        if is_na_percent > 1:
            print(f"{year} NaNs: {is_na_percent}% - {col}")

In [10]:
df.to_csv('data/clean_metrics_18_22_v1.0.csv', index=False)